In [ ]:
import json

with open("essential_account.json","r",encoding="UTF-8") as file:
    existed_data = json.load(file)
    accounts = existed_data["essential_accounts"]

for account in accounts:
    print(account)
    account["name"]




In [ ]:
import pandas as pd 

from src import GameMaster, Player

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager
product_A = game_master.construct_instance("inventory","product_A").get("ID")
bilding_B = game_master.construct_instance("building", "building_B", address="東京都品川区荏原3-7-13").get("ID")
asset_list = list(game_master.asset_registry.keys())
player1.redister_product(product_A)
player1.aquire_building(bilding_B, 2000)

player1.purchase_product(product_A, 100, 150)
player1.purchase_product(product_A, 200, 120, 15)

player1.sale_product(product_A, 80)

player1.purchase_product(product_A, 30, 160)
player1.sale_product(product_A, 100, 225)

player1.sale_product(product_A, 100, 210, 500)

player1.perform_inventory_audit(product_A)

end_1 = ledger1.execute_settlement()

ledger1.display_transaction_history()
# ledger1.display_trial_balance(end_1)
# ledger1.display_financial_statements(end_1)

df = pd.DataFrame.from_dict(end_1, orient="index")


AttributeError: partially initialized module 'scripts.player' has no attribute 'GameMaster' (most likely due to a circular import)

In [ ]:
ledger1.display_financial_statements(end_1)

In [ ]:
inventory_c = game_master.construct_instance("inventory", "inventory_c")
type(inventory_c.get("class"))


In [7]:
import pandas as pd 

from player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager

building_B = game_master.construct_instance(
    asset_type="building",
    name="building2",
    value=1000000,
    address="tokyo"
    )

資産 'building2' (ID: 245aed5f-224e-4af4-ae73-a8e3a7f2b71d, クラス: <class 'scripts.asset.Building'>) が登録されました。


In [12]:
from src import *
 # ゲームマスターを初期化
game_master = GameMaster()

# プレイヤーの初期化・登録
player1 = Player(name="Player1", game_master=game_master, initial_cash= 2000000)
player2 = Player(name="Player2", game_master=game_master, initial_cash= 2000000)
game_master.players.append(player1)
game_master.players.append(player2)

# 資産の生成と登録
building_1 = game_master.construct_instance(
    asset_type="building",
    name="OfficeBuilding_1",
    value=1000000,
    address="東京都新宿区早稲田鶴巻町"
)["ID"]
building_2 = game_master.construct_instance(
    asset_type="building",
    name="OfficeBuilding_2",
    value=1500000,
    address="東京都品川区荏原"
)["ID"]
product_A = game_master.construct_instance(
    asset_type="inventory",
    name="product_A"
)["ID"]
product_B = game_master.construct_instance(
    asset_type="inventory",
    name="product_B"
)["ID"]

# プレイヤーに資産を取得させる
player1.aquire_building(asset_id=building_1, value=1000000)
player2.aquire_building(asset_id=building_2, value=1500000)

# プレイヤーに商品を登録させる
player1.redister_product(product_id=product_A)
player2.redister_product(product_id=product_B)

# 商品を仕入れ、売り上げる
player1.purchase_product(product_id=product_A, quantity= 500, price= 120)
player2.purchase_product(product_id=product_B, quantity= 600, price= 80)

player1.sale_product(product_id=product_A, quantity= 320, sales_price= 300)
player2.sale_product(product_id=product_B, quantity= 500, sales_price= 210)

player1.sale_product(product_id=product_A, quantity= 150, sales_price =250)
player2.sale_product(product_id=product_B, quantity= 80) # 売価の更新なし

# ゲーム内時間を90日進める
print("\n=== 時間を365日進めます ===")
game_master.advance_time(days=365)


資産 'OfficeBuilding_1' (ID: 8e8cc3c6-62bf-4b21-b965-1f928d16ae2b, クラス: <class 'scripts.asset.Building'>) が登録されました。
資産 'OfficeBuilding_2' (ID: ff036c60-db52-4767-9f47-360b98cc2866, クラス: <class 'scripts.asset.Building'>) が登録されました。
資産 'product_A' (ID: 44f094aa-d071-403e-9d64-dfa4dbb11352, クラス: <class 'scripts.asset.Inventory'>) が登録されました。
資産 'product_B' (ID: 18944765-8c19-47b1-a41f-92aacbe3423a, クラス: <class 'scripts.asset.Inventory'>) が登録されました。
OfficeBuilding_1 の所有者が Player1 に設定されました。
Building instance type: <class 'scripts.asset.Building'>
OfficeBuilding_2 の所有者が Player2 に設定されました。
Building instance type: <class 'scripts.asset.Building'>
[Player1]**商品が登録されました** 商品名：product_A
[Player2]**商品が登録されました** 商品名：product_B
[Player1,product_A]**売価が更新されました**　更新後：300
[Player2,product_B]**売価が更新されました**　更新後：210
[Player1,product_A]**売価が更新されました**　更新後：250

=== 時間を365日進めます ===
イベント記録: {'date': '2024-12-31', 'event': '365日進行', 'details': {}}
OfficeBuilding_1 の減価償却が適用されました: 25000 減価償却累計額: 25000 残存価額: 975000
[Playe

In [13]:
# 資産の状態を表示
print("\n=== 資産の状態 ===")
for player in game_master.players:
    for asset_info in player.portfolio:
        asset_obj = game_master.get_asset_by_id(asset_info["ID"])
        if isinstance(asset_obj, asset.Tangible):
            print(f"プレイヤー名: {player.name}, 資産名: {asset_obj.name}, 帳簿価額: {int(asset_obj.value)}, 累計減価償却額: {int(asset_obj.accumulated_depreciation)}")
        if isinstance(asset_obj, asset.Inventory):
            print(f"プレイヤー名: {player.name}, 資産名: {asset_obj.name}, 帳簿価額: {int(asset_obj.value)}")
        


=== 資産の状態 ===
プレイヤー名: Player1, 資産名: OfficeBuilding_1, 帳簿価額: 975000, 累計減価償却額: 25000
プレイヤー名: Player1, 資産名: product_A, 帳簿価額: 3600
プレイヤー名: Player2, 資産名: OfficeBuilding_2, 帳簿価額: 1462500, 累計減価償却額: 37500
プレイヤー名: Player2, 資産名: product_B, 帳簿価額: 1600


In [ ]:
# 勘定元帳の記録を確認
print("\n=== プレイヤー1の勘定元帳 ===")
player1.ledger_manager.display_transaction_history()

In [ ]:
# 勘定元帳の記録を確認
print("\n=== プレイヤー2の勘定元帳 ===")
player2.ledger_manager.display_transaction_history()

In [ ]:
end_1 = [player1.ends[0],player2.ends[0]]
player1.ledger_manager.display_financial_statements(end_1[0])

In [ ]:
player2.ledger_manager.display_financial_statements(end_1[1])

In [ ]:
from src.player import *

# ゲームマスターを初期化
game_master = GameMaster()

# プレイヤーを作成して登録
player = Player(name="Player1", game_master=game_master)
game_master.players.append(player)

# 資産の生成と登録
building_asset = game_master.construct_instance(
    asset_type="building",
    name="Office Building",
    value=1000000,
    address="Downtown City"
)

# プレイヤーに建物資産を取得させる
player.aquire_building(asset_id=building_asset["ID"], value=1000000)

# ゲーム内時間を進める
print("\n=== 90日進行 ===")
game_master.advance_time(days=90)

# 資産の確認
print("\n=== プレイヤーのポートフォリオ ===")
for asset_info in player.portfolio:
    asset = asset_info.get("instance")
    print(f"資産名: {asset.name}, 帳簿価額: {int(asset.value)}, 累計減価償却額: {int(asset.accumulated_depreciation)}")

# 勘定元帳の記録を確認
print("\n=== プレイヤーの勘定元帳 ===")
player.ledger_manager.display_transaction_history()

In [22]:
import sqlalchemy    
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

import json

Base = declarative_base()
Engine = create_engine("sqlite:///test.sqlite3",
                       echo=True)
Session = sessionmaker(bind=Engine)
session = Session()

class account(Base):
    __tablename__ = "account"
    id = Column(Integer, primary_key=True)
    name = Column(String(63), unique=True)
    statement = Column(String(63))
    category = Column(String(63))
    
    
# Drop the existing table if it exists
try:
    Base.metadata.drop_all(Engine)
    existed_data = session.query(account).all()
    for row in existed_data:
        session.delete(row)
except:
    pass

# Create the table with the new schema
Base.metadata.create_all(Engine)

with open("database/essential_account.json","r",encoding="UTF-8") as file:
    data = json.load(file)
    aaa = data["essential_accounts"]
    # Filter out duplicates
    unique_accounts = {item["name"]: item for item in aaa}.values()
    session.add_all([account(name=item["name"], statement=item["statement"], category=item["category"]) for item in unique_accounts])
session.commit()


session.close()



2025-01-08 22:14:01,877 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-08 22:14:01,878 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("account")
2025-01-08 22:14:01,879 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-01-08 22:14:01,881 INFO sqlalchemy.engine.Engine 
DROP TABLE account
2025-01-08 22:14:01,881 INFO sqlalchemy.engine.Engine [no key 0.00091s] ()
2025-01-08 22:14:01,925 INFO sqlalchemy.engine.Engine COMMIT
2025-01-08 22:14:01,928 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-08 22:14:01,929 INFO sqlalchemy.engine.Engine SELECT account.id AS account_id, account.name AS account_name, account.statement AS account_statement, account.category AS account_category 
FROM account
2025-01-08 22:14:01,930 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ()
2025-01-08 22:14:01,932 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-01-08 22:14:01,932 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("account")
2025-01-08 22:14:01,932 INFO sqlalchemy.engin

C:\Users\hacch\AppData\Local\Temp\ipykernel_24572\3783427201.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
